#### Challenge 42: Bleichenbacher's e=3 RSA Attack

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)

In [4]:
from Crypto.Util import number
from Crypto.Random import random
from Crypto.Hash.SHA256 import SHA256Hash

import cryptopals as cp

<div class="alert alert-block alert-info">

<div class="alert alert-block alert-warning">    

#### **Crypto-tourism informational placard.**

This attack broke Firefox's TLS certificate validation several years ago. You could write a Python script to fake an RSA signature for any certificate. We find new instances of it every other year or so.

</div>
    
RSA with an encrypting exponent of 3 is popular, because it makes the RSA math faster.

With `e=3` RSA, encryption is just cubing a number mod the public encryption modulus:

`c = m ** 3 % n`

    
`e=3` is secure as long as we can make assumptions about the message blocks we're encrypting. The worry with low-exponent RSA is that the message blocks we process won't be large enough to wrap the modulus after being cubed. The block `00:02` (imagine sufficient zero-padding) can be "encrypted" in `e=3` RSA; it is simply `00:08`.

When RSA is used to sign, rather than encrypt, the operations are reversed; the verifier "decrypts" the message by cubing it. This produces a "plaintext" which the verifier checks for validity.

When you use RSA to sign a message, you supply it a block input that contains a message digest. The PKCS1.5 standard formats that block as:

`00h 01h ffh ffh ... ffh ffh 00h ASN.1 GOOP HASH`

As intended, the ffh bytes in that block expand to fill the whole block, producing a "right-justified" hash (the last byte of the hash is the last byte of the message).

There was, 7 years ago, a common implementation flaw with RSA verifiers: they'd verify signatures by "decrypting" them (cubing them modulo the public exponent) and then "parsing" them by looking for `00h 01h ... ffh 00h ASN.1 HASH`.

This is a bug because it implies the verifier isn't checking all the padding. If you don't check the padding, you leave open the possibility that instead of hundreds of ffh bytes, you have only a few, which if you think about it means there could be squizzilions of possible numbers that could produce a valid-looking signature.

How to find such a block? Find a number that when cubed (a) doesn't wrap the modulus (thus bypassing the key entirely) and (b) produces a block that starts `"00h 01h ffh ... 00h ASN.1 HASH"`.

There are two ways to approach this problem:

- You can work from Hal Finney's writeup, available on Google, of how Bleichenbacher explained the math "so that you can do it by hand with a pencil".
- You can implement an integer cube root in your language, format the message block you want to forge, leaving sufficient trailing zeros at the end to fill with garbage, then take the cube-root of that block.

Forge a 1024-bit RSA signature for the string "hi mom". Make sure your implementation actually accepts the signature!

</div>

In [5]:
key_size = 1024
[e,d,n] = cp.genRSA_keypair(key_size)
m = int((b'This is ridiculous! There\'s no such thing as privacy anymore!'*4).hex(), 16)
c = pow(m, e, n)

In [12]:
s = 79
c_ = (pow(s, e, n) * c) % n
p_ = pow(c_, d, n)

Despite the warning, I wanted to try straight division (not multiplying by the inverse mod N):

In [13]:
recovered_pt = (p_ // s) % n

print(bytes.fromhex(hex(recovered_pt)[2:]).decode())

This is ridiculous! There's no such thing as privacy anymore!This is ridiculous! There's no such thing as privacy anymore!This is ridiculous! There's no such thing as privacy anymore!This is ridiculous! There's no such thing as privacy anymore!


Now the way it says to do it:

In [14]:
mi = cp.invmod(s, n)
recovered_pt = (p_*mi) % n
print(bytes.fromhex(hex(recovered_pt)[2:]).decode())

This is ridiculous! There's no such thing as privacy anymore!This is ridiculous! There's no such thing as privacy anymore!This is ridiculous! There's no such thing as privacy anymore!This is ridiculous! There's no such thing as privacy anymore!


Let's try to break the division approach:

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)

In [44]:
key_size = 1024
for ii in range(1000):
    
    print('.')
    [e,d,n] = cp.genRSA_keypair(key_size)
    c = pow(m, e, n)
    s = random.randint(0, 2**128-1)
    c_ = (pow(s, e, n) * c) % n
    p_ = pow(c_, d, n)
    recovered_pt = (p_ // s) % n
    
    if recovered_pt != m:        
        print('Broke it!')
        print(f'e={e}\nd={d}\nn={n}\ns={s}\n')
        break


.
Broke it!
e=3
d=13784155787454640965203320918456828855576350957154037222797011255165358470367881823584041715061754827774338504096176436120475191939229449781152894319605928252913300736282366016848779083912182646827542663181074178251488178035165720461277396960286993903600420480331825348205323534468578466963075371691051312181999979240211075743135335667166862700410731000271406232507052029364966540907676351362097390400954839239373713919572846050908107205998352793777885820373061841666941478495929055051905449803604551103700362868106112893945484374599667847712517441128352550514766396104086948998499666483037133089815898176502824916123
n=20676233681181961447804981377685243283364526435731055834195516882748037705551822735376062572592632241661507756144264654180712787908844174671729341479408892379369951104423549025273168625868273970241313994771611267377232267052748580691916095440430490855400630720497738022307985301702867700444613057536576968273287661965976930946948637139294611572506276884739532485

Here are two trivial examples of where division by a # and multiplication by a multiplicative inverse are 1) the same, and 2) different within a cyclic group (modulo arithmetic).

In [43]:
n = 17
s = 4
x = 12

print((x//s) % n)
print((x * cp.invmod(s, n)) % n)

n = 17
s = 4
x = 9

print((x//s) % n)
print((x * cp.invmod(s, n)) % n)

3
3
2
15
